In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchsummary import summary
import torch.optim as optim

import matplotlib.pyplot as plt

from utils.data import Data
from utils.model import ResNet
from utils.training import Training

In [2]:
BATCH_SIZE = 256
VAILD_RATIO = 0.9
LR = 0.1
EPOCHS = 200

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Data Preparation

In [3]:
data = Data(batch_size = BATCH_SIZE, valid_ratio = VAILD_RATIO)

train_transforms = transforms.Compose([
                           transforms.RandomHorizontalFlip(),
                           transforms.RandomCrop(32, padding = 4),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = data.means, 
                                                std = data.stds)
                       ])

train_iterator = data.getTrainData(train_transforms)
test_iterator = data.getTestData()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data
Files already downloaded and verified
Files already downloaded and verified


## Modeling

In [4]:
model = ResNet([2, 2, 2]).to(device)
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
     ResidualBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
    ResidualBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

t = Training(
    device = device,
    model = model, 
    optimizer = optimizer, 
    criterion = criterion,
    scheduler = scheduler,
    epochs = EPOCHS)

t.start_training(train_iterator, logFileName = 'FINAL')

epoch:1 | Train Loss: 2.347 | Train Acc: 24.388%
epoch:2 | Train Loss: 1.688 | Train Acc: 36.218%
epoch:3 | Train Loss: 1.519 | Train Acc: 43.123%
epoch:4 | Train Loss: 1.378 | Train Acc: 49.117%
epoch:5 | Train Loss: 1.233 | Train Acc: 55.047%
epoch:6 | Train Loss: 1.084 | Train Acc: 60.937%
epoch:7 | Train Loss: 0.974 | Train Acc: 65.062%
epoch:8 | Train Loss: 0.879 | Train Acc: 68.869%
epoch:9 | Train Loss: 0.800 | Train Acc: 71.849%
epoch:10 | Train Loss: 0.722 | Train Acc: 74.649%
epoch:11 | Train Loss: 0.664 | Train Acc: 76.587%
epoch:12 | Train Loss: 0.597 | Train Acc: 79.053%
epoch:13 | Train Loss: 0.547 | Train Acc: 81.162%
epoch:14 | Train Loss: 0.510 | Train Acc: 82.365%
epoch:15 | Train Loss: 0.473 | Train Acc: 83.685%
epoch:16 | Train Loss: 0.444 | Train Acc: 84.709%
epoch:17 | Train Loss: 0.428 | Train Acc: 85.244%
epoch:18 | Train Loss: 0.404 | Train Acc: 86.064%
epoch:19 | Train Loss: 0.391 | Train Acc: 86.600%
epoch:20 | Train Loss: 0.368 | Train Acc: 87.332%
epoch:21 

In [6]:
t.test(test_iterator)

Test Loss: 0.176 | Test Acc: 94.834%
